In [19]:
import pyodbc
import sqlite3 as sql
import pandas as pd
import warnings
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

warnings.simplefilter('ignore')

Load in .env file for safe connection

In [20]:
class EnvConfig:
    def __init__(self):
        load_dotenv()
    
    def get(self, key: str) -> str:

        value = os.getenv(key)
        if not value:
            print(f"Warning: {key} not found in environment")
        return value
    
    def is_configured(self) -> bool:
        required_keys = ["DB_SERVER", "DB_NAME_SDM", "DB_NAME_DWH", "DB_USER", "DB_PASSWORD"]
        return all(self.get(key) for key in required_keys)

Safe connection

In [33]:
env = EnvConfig()

SDM_CONN_PATH = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={env.get('DB_SERVER')},1433;"
    f"DATABASE={env.get('DB_NAME_SDM')};"
    f"UID={env.get('DB_USER')};"
    f"PWD={env.get('DB_PASSWORD')}"
)

DWH_CONN_PATH = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={env.get('DB_SERVER')},1433;"
    f"DATABASE={env.get('DB_NAME_DWH')};"
    f"UID={env.get('DB_USER')};"
    f"PWD={env.get('DB_PASSWORD')}"
)

FROM_DB = pyodbc.connect(SDM_CONN_PATH)
SDM_cursor = FROM_DB.cursor()

TO_DB = pyodbc.connect(DWH_CONN_PATH)
DWH_cursor = TO_DB.cursor()

Dataframes

In [22]:
def create_dataframes_sql(connection):
    dictionary : dict = {}
    query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
    key = "TABLE_NAME"
    
    tables = pd.read_sql(query, connection)
    
    for table in tables[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
    
    return dictionary

sdm_frames = create_dataframes_sql(FROM_DB)

Query's

In [23]:
from typing import Iterable

def query_remove(table_name : str):
    try:
        query = f"DELETE FROM {table_name}"
        DWH_cursor.execute(query)
        DWH_cursor.commit()
    except pyodbc.Error as e:
        print(f"ERROR: {table_name}: {e}")
        return table_name
    
    return None

def right_type(value, column_name, types):
    dtype = types[column_name]
    
    if pd.isna(value):
        return "NULL"
    
    if dtype == "object" or dtype == "string":
        value = value.replace("'", "''")
        return f"'{value}'"
    
    return f"{value}"
    

def create_add_query(row, types):
    query = ""
    columns = list(row.keys())
    
    for pos in range(len(columns)):
        column_name = columns[pos]
        value = row[column_name]
        
        if (pos == len(columns) - 1):
            query += f"{column_name}) VALUES ("
        else:
            query += f"{column_name}, "
    
    data = list(row)
    
    for pos in range(len(data)):
        column_name = columns[pos]
        value = row[column_name]
        
        if pos == len(columns) - 1:
            query += f"{right_type(value, column_name, types)})"
        else:
            query += f"{right_type(value, column_name, types)}, "
    
    return query

def query_add(table_name : str, table_data : pd.DataFrame):
    queries = []
    types = table_data.dtypes

    for index, row in table_data.iterrows():
        query : str = f"INSERT INTO {table_name} ("
        query += create_add_query(row, types)
        queries.append(query)
    
    return queries

Maak het DWH leeg

In [24]:
def empty_dwh():
    tables_dwh : list = [
        "DimProduct",
        "DimCustomer",
        "DimEmployee",
        "DimDate",
        "DimTerritory",
        "FactSales",
    ]

    while len(tables_dwh) > 0:
        for table in tables_dwh:
            try:
                temp = query_remove(table)
                
                if (temp == None):
                    tables_dwh.remove(table)
                    print(f"REMOVED {table}")
                else:
                    print(f"NOT YET REMOVED: {table}")
            except pyodbc.Error as e:
                print(e)
        
    print("All items are removed")


Datum converter

In [25]:
def convert_date(date : str):
    converted = pd.to_datetime(date)
    year = converted.year
    month = converted.month
    quarter = (month - 1 ) // 3 + 1
    converted = converted.strftime("%Y-%m-%d")
    
    return pd.DataFrame({
        "Date" : [converted], 
        "Year" : [year], 
        "Month" : [month], 
        "Quarter" : [quarter]}).astype({
            "Year": "Int64",
            "Month" : "Int64",
            "Quarter" : "Int64"
        })

Dataframes aanmaken

In [30]:
dates_dwh = pd.DataFrame({
    "Date" : [],
    "Year" : [],
    "Quarter" : [],
    "Month" : []
})

def create_territory():
    dim_territory = sdm_frames["Sales_SalesTerritory"].copy()

    dim_territory = dim_territory[["TerritoryID", "Name", "Group"]]

    return dim_territory


def create_product():

    dim_product = sdm_frames["Production_Product"].copy()

    sdm_frames["Suppliers"]["SupplierID"] = sdm_frames["Suppliers"]["SupplierID"].astype(str)
    dim_product["SupplierID"] = dim_product["SupplierID"].astype(str)

    dim_product = pd.merge(dim_product, sdm_frames["Suppliers"], on="SupplierID", how="outer")

    dim_product = dim_product[[
        "ProductID","NWProductID", "Name", "description", "Category", "Color", "CompanyName", "Country", "ListPrice"
    ]].rename(columns={
        "Country": "SupplierCountry",
        "description": "Description",
    })

    dim_product = dim_product[~(dim_product["ProductID"].isna() & dim_product["NWProductID"].isna())]

    return dim_product


def create_employee():
    global dates_dwh

    dim_employee = sdm_frames["Employee"].copy()

    dim_employee["EmpFullName"] = dim_employee["emp_fname"] + " " + dim_employee["emp_lname"]

    dim_employee = dim_employee[["emp_id", "manager_id", "EmpFullName", "Title", "start_date", "dept_id"]].rename(columns={
                                   "emp_id" : "EmpID",
                                   "manager_id" : "ManagerID",
                                   "dept_id" : "DeptID",
                                   "start_date" : "StartDate"
    })

    for index, row in dim_employee.iterrows():
        date_details : pd.DataFrame = convert_date(row["StartDate"])
        dates_dwh = pd.concat([dates_dwh, date_details], ignore_index=True)

    return dim_employee

def create_customer():
    dim_customer = sdm_frames["Customer"].copy()

    dim_customer["FullName"] = dim_customer["fname"] + " " + dim_customer["lname"]

    dim_customer = dim_customer[[
        "id", "FullName", "address", "company_name"
    ]].rename(columns={
        "id" : "CustomerID",
        "address": "Address",
        "company_name": "CompanyName"
    })

    dim_customer = dim_customer.drop_duplicates()

    return dim_customer



territory_dwh, customer_dwh, employee_dwh, product_dwh= (
    create_territory(),
    create_customer(),
    create_employee(),
    create_product()
    )


def create_fact_sales ():
    global dates_dwh
    fact_sales = pd.merge(sdm_frames["Sales_SalesOrderDetail"], sdm_frames["Sales_SalesOrderHeader"], on="SalesOrderID")

    product_cost = sdm_frames["Production_Product"][["ProductID", "StandardCost"]]
    fact_sales = pd.merge(fact_sales, product_cost, on="ProductID", how="left")

    fact_sales["TotalSales"] = (fact_sales["UnitPrice"] * fact_sales["OrderQty"]) - fact_sales["UnitPriceDiscount"]
    fact_sales["TotalProfits"] = ((fact_sales["UnitPrice"] - fact_sales["StandardCost"]) * fact_sales["OrderQty"]) - fact_sales["UnitPriceDiscount"]

    fact_sales = fact_sales[[
        "SalesOrderID",
        "SalesOrderDetailID",
        "CustomerID",
        "SalesCustomerID",
        "SalesPersonID",
        "ProductID",
        "TerritoryID",
        "OrderDate",
        "UnitPrice",
        "UnitPriceDiscount",
        "OrderQty",
        "TotalSales",
        "TotalProfits",
        "NWProductID",
        "StandardCost"
        ]].rename(columns= {
            "StandardCost" : "Costs",
            "SalesPersonID" : "EmpID",
            "TotalProfits" : "TotalProfit"
                            })
    
    for index, row in fact_sales.iterrows():
        date_details : pd.DataFrame = convert_date(row["OrderDate"])
        dates_dwh = pd.concat([dates_dwh, date_details], ignore_index=True)

    return fact_sales

fact_sales_dwh = create_fact_sales()
dates_dwh = dates_dwh.drop_duplicates()


In [ ]:
emp_id_to_check = 276

exists = emp_id_to_check in employee_dwh['EmpID'].values

print(f"EmpID {emp_id_to_check} exists: {exists}")

In [ ]:
sorted_df = sdm_frames["Sales_SalesOrderHeader"].sort_values(by="SalesPersonID", ascending=False)
print(sorted_df["SalesPersonID"])

In [17]:
territory_dwh

,TerritoryID,Name,Group
0,1,Northwest,North America
1,2,Northeast,North America
2,3,Central,North America
3,4,Southwest,North America
4,5,Southeast,North America
5,6,Canada,North America
6,7,France,Europe
7,8,Germany,Europe
8,9,Australia,Pacific
9,10,United Kingdom,Europe


In [34]:
empty_dwh()

tables = {
    "DimDate": dates_dwh,
    "DimProduct": product_dwh,
    "DimCustomer": customer_dwh,
    "DimEmployee": employee_dwh,
    "DimTerritory": territory_dwh,
    "FactSales": fact_sales_dwh
}


def format_value(value, column_name, table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name', 'state']

    if column_name.lower() in reserved_keywords:
        column_name = f"[{column_name}]"

    if pd.isna(value):
        return "NULL"

    if isinstance(value, bool):
        return "1" if value else "0"

    if column_name.lower() in ['phone', 'zip_code']:
        return f"'{str(value)}'"
    
    if column_name.lower() == "customerid" and table_name.lower() != "Sales_Customer":
        return f"'{str(value)}'"
    
    elif isinstance(value, str):
        if value == "":
            return "NULL"
        return f"'{value.replace("'", "''")}'"
    
    elif isinstance(value, pd.Timestamp):
        return f"'{value.strftime('%Y-%m-%d')}'"
    
    elif isinstance(value, bytes):
        hex_value = value.hex()
        return f"0x{hex_value}"
    
    elif isinstance(value, (int, float)):
        return str(value)
    
    return f"'{str(value)}'"

def format_table_name(table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'state']

    if table_name.lower() in reserved_keywords:
        return f"[{table_name}]"
    
    return table_name 

for table_name, df in tables.items():

    error_count = 0
    total_count = 0

    try:
        print(f"Loading table {table_name}:", end=" ")

        formatted_table_name = format_table_name(table_name)

        for index, row in df.iterrows():
            total_count += 1

            columns = df.columns.tolist()

            if 'rowguid' in columns:
                columns.remove('rowguid')

            formatted_columns = [
                f"[{col}]" if col.lower() in ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name'] else col
                for col in columns
            ]

            values = []
            for col in columns:
                value = row[col]
                values.append(format_value(value, col, table_name))

            column_names = ", ".join(formatted_columns)
            value_string = ", ".join(values)

            query = f"INSERT INTO {formatted_table_name} ({column_names}) VALUES ({value_string})"
            try:
                DWH_cursor.execute(query)
            except pyodbc.Error as e:
                print(query)
                print(e)
                error_count += 1
                continue
            
        TO_DB.commit()
    

        passed_count = total_count - error_count

        if error_count == 0:
            print(f"100% ✓")
        else:
            print(f"{passed_count}/{total_count} rows ✓")

    except pyodbc.Error as e:
        print("✗ ERROR")
        print(f"{query}")
        print(f"Error message: {e}")
        print("-" * 80)



REMOVED DimProduct
REMOVED DimEmployee
REMOVED DimTerritory
REMOVED DimCustomer
REMOVED FactSales
REMOVED DimDate
All items are removed
Loading table DimDate: 100% ✓
Loading table DimProduct: 100% ✓
Loading table DimCustomer: 100% ✓
Loading table DimEmployee: 100% ✓
Loading table DimTerritory: 100% ✓
Loading table FactSales: 100% ✓
